In [1]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# from google.colab import drive
# drive.mount("/content/gdrive")
# pre_path = 'gdrive/My Drive/AN2DL/HW2/'
# %cd gdrive/My Drive/AN2DL/HW2/

In [5]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import cv2
from statsmodels.tsa.stattools import acf

# Setting seeds for reproducibility
seed = 2023
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Suppressing unnecessary warnings and logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

print(tf.__version__)

# Additional imports for the model
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D, Activation,Input,Add


2.15.0


In [2]:
X_train = np.load('windowed_training_data/train_X.npy',allow_pickle='TRUE').item()
y_train = np.load('windowed_training_data/train_y.npy',allow_pickle='TRUE').item()

X_validation = np.load('windowed_training_data/validation_X.npy',allow_pickle='TRUE').item()
y_validation = np.load('windowed_training_data/validation_y.npy',allow_pickle='TRUE').item()

#X_test = np.load(pre_path +'windowed_training_data/test_X.npy',allow_pickle='TRUE').item()
#y_test = np.load(pre_path +'windowed_training_data/test_y.npy',allow_pickle='TRUE').item()

In [3]:
window = 206
output_shape = 18
training_categories = ['A', 'B', 'C', 'D','E', 'F']
X_train = np.concatenate([X_train[i] for i in training_categories]).reshape(-1,window,1)
y_train = np.concatenate([y_train[i] for i in training_categories])

X_validation = np.concatenate([X_validation[i] for i in training_categories]).reshape(-1,window,1)
y_validation = np.concatenate([y_validation[i] for i in training_categories])

#X_test = np.concatenate([X_test[i] for i in training_categories]).reshape(-1,window,1)
#y_test = np.concatenate([y_test[i] for i in training_categories])

In [4]:
model = load_model('models/one_hat_labeled_normalised_16_32_32_64_128_256_dense.h5')
batch_size = 2**10
epochs = 30
with tf.device('gpu'):
    history = model.fit(
        x = X_train,
        y = y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_data=(X_validation, y_validation),
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=6, restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=4, factor=0.9, min_lr=1e-7)
        ]
    ).history

2023-12-19 17:42:09.095989: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1228268408 exceeds 10% of free system memory.


Epoch 1/30
 114/1456 [=>............................] - ETA: 12:33 - loss: 0.0089

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save('models/trained_30_one_hat_labeled_normalised_16_32_32_64_128_256_dense.h5')